# Fresh take

Trying to get this right.  2024/02/18

## Setup an inventory

In [381]:
import os
import yaml
import copy
import shutil
import numpy as np
import pandas as pd
import importlib
import NewDev as ND
import helperFunctions as hf
importlib.reload(ND)
importlib.reload(hf)
                          
dbpth = os.path.join(os.getcwd(),'test')
shutil.rmtree(dbpth)
os.mkdir(dbpth)

# db = ND.database(dbpth,verbose=True)

ND.site(siteID='SCL',description='Swiss Cheese Lake Eddy Covariance Station',
                latitude='N 69 13.5850',longitude='W 135 15.1144',
                PI = 'June Skeeter, Peter Morse',dateEstablished='2024-07-20').entry
# db._siteInventory(siteID='SCL',measurementID='TS',description='Salix Sedge Meadow',
#                   latitude='N 69 13.5239',longitude='W 135 15.1358')
# db._siteInventory(siteID='SCL',measurementID='TS',description='Wet Sedge Meadow',
#                   latitude='N 69 13.5521',longitude='W 135 15.1350',overwrite=True)

# db.siteInventory.siteInventory


{'SCL-.site-1': {'siteID': 'SCL',
  'measurementID': '.site',
  'replicateID': 1,
  'description': 'Swiss Cheese Lake Eddy Covariance Station',
  'name': None,
  'PI': 'June Skeeter, Peter Morse',
  'dateEstablished': '2024-07-20',
  'landCoverType': None,
  'longitude': -135.2519067,
  'latitude': 69.2264167}}

## File Management

In [352]:

source = r'..\SCL_2024'
# print(source)
for dir,_,files in os.walk(source):
    # print(dir)
    for f in files:
        if f.endswith('.csv'):
            file = os.path.join(dir,f)
            for key in ['WSM']:
                if key in file:
                    fn = file

In [363]:
# fn

import rawDataFile as rawDataFile
importlib.reload(rawDataFile)

rd = rawDataFile.hoboCSV(source=fn,verbose=False,excludePattern={r"LGR(.*?)LBL: ":'',
                                                   r'\)':'cm'})
rd.Metadata

{'verbose': False,
 'loggerName': None,
 'timeZone': 'UTC',
 'frequency': '30min',
 'yearfirst': True,
 'Variables': {'_': {'siteID': '.example',
   'measurementID': '.site',
   'replicateID': 1,
   'description': None,
   'name': None,
   'PI': None,
   'dateEstablished': None,
   'landCoverType': None,
   'longitude': None,
   'latitude': None,
   'ignore': True,
   'originalName': '#',
   'unit': None,
   'dtype': '<i8',
   'variableDescription': None,
   'V': None,
   'H': None,
   'R': None,
   'variableID': '_'},
  'Date_Time_GMT_00_00': {'siteID': '.example',
   'measurementID': '.site',
   'replicateID': 1,
   'description': None,
   'name': None,
   'PI': None,
   'dateEstablished': None,
   'landCoverType': None,
   'longitude': None,
   'latitude': None,
   'ignore': True,
   'originalName': 'Date Time, GMT+00:00',
   'unit': None,
   'dtype': '|O',
   'variableDescription': None,
   'V': None,
   'H': None,
   'R': None,
   'variableID': 'Date_Time_GMT_00_00'},
  'Temp_C_5c

# Database

In [322]:
importlib.reload(ND)

# dbpth = os.path.join(os.getcwd(),'test2')
# # if os.path.isdir(dbpth):
# #     shutil.rmtree(dbpth)

# db = pyDB.database(projectPath=dbpth)
# # db = pyDB.database(projectPath=dbpth,
# #     # siteID=['SCL'],
# #     # Years=[str(y) for y in range(2024,2025)],
# #     # verbose=False
# #     )
# # db.
n = ND.rawDatabaseImport(projectPath=dbpth,measurementID ='test',dataIn=rd.Data,metadataIn=rd.Metadata)

n

_ {'ignore': True, 'name_in': '#', 'unit_in': None, 'safe_name': '_', 'dtype': '<i8', 'variableDescription': None}
Date_Time_GMT_00_00 {'ignore': True, 'name_in': 'Date Time, GMT+00:00', 'unit_in': None, 'safe_name': 'Date_Time_GMT_00_00', 'dtype': '|O', 'variableDescription': None}
Temp_C_5cm {'ignore': False, 'name_in': 'Temp, °C (5cm', 'unit_in': None, 'safe_name': 'Temp_C_5cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_25cm {'ignore': False, 'name_in': 'Temp, °C (25cm', 'unit_in': None, 'safe_name': 'Temp_C_25cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_50cm {'ignore': False, 'name_in': 'Temp, °C (50cm', 'unit_in': None, 'safe_name': 'Temp_C_50cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_100cm {'ignore': False, 'name_in': 'Temp, °C (100cm', 'unit_in': None, 'safe_name': 'Temp_C_100cm', 'dtype': '<f8', 'variableDescription': None}


rawDatabaseImport(projectPath='c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\test', overwrite=False, verbose=True, logFile='Creating Database: 2025-03-01 21:22:42\n', metadataFile={'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'POSIX_timestamp': {'dtype': 'float64', 'timezone': 'UTC', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time'}}, 'Date_created': '2025-03-01 21:22:42', 'Last_modified': '2025-03-01 21:22:42'}, measurementID='test', dataIn=                        _ Date_Time_GMT_00_00  Temp_C_5cm  Temp_C_25cm  \
2024-07-20 21:00:00     1   24/07/20 21:00:00       7.217        2.423   
2024-07-20 21:30:00     2   24/07/20 21:30:00      11.759        6.712   
2024-07-20 22:00:00     3   24/07/20 22:00:00      11.467        5.257   
2024-07-20 22:30:00     4   24/07/20 22:30:00      11.029        4.428   
2024-07-20 23:00:00     5   24/0

In [61]:
siteID,subsiteID='a'.split('-',1)

ValueError: not enough values to unpack (expected 2, got 1)